In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import json
import argparse
import time

import build_time_dist_matrix
import optimization
import visualization
import schedule
import upload_results
import manual_viz



with open('data/config.json', 'r') as opened:
    config = json.load(opened)

In [ ]:

    
print(' *   Building Time/Distance Matrices')
#check if node_loader_options are specified
if 'node_loader_options' in config.keys():
    node_data = build_time_dist_matrix.process_nodes(config['node_loader_options'], config['zone_configs'])
else:
    node_data = build_time_dist_matrix.process_nodes()

print(f' *   Starting Model Run at {time.strftime("%H:%M:%S")} (UTC)')

routes_for_mapping_viz, vehicles_viz, zone_route_map = optimization.main(node_data, config)

visualization.main(routes_for_mapping_viz, vehicles_viz, zone_route_map)



In [ ]:
import datetime
import matplotlib
from matplotlib import pyplot
import numpy as np
import pandas as pd

In [ ]:
pyplot.rcParams["figure.figsize"] = [3,7]
for route in route_dict:
    distances = [0]+route_dict[route]['current_distance'][:-1]
    times = route_dict[route]['time']
    distances = np.array(distances).cumsum()
    times = np.array(times)

    y_ticks = [f'{name} - {datetime.timedelta(seconds=float(one_time))}' for one_time,name in zip(times,route_dict[route]['current_names'])]
    pyplot.yticks(ticks=times, labels=y_ticks)
    pyplot.plot(distances, times, marker='o')
    pyplot.show()

In [ ]:
pyplot.rcParams["figure.figsize"] = [3,7]
for route in route_dict:
    distances = pd.Series(route_dict[route]['travel_time']).shift(1).values[1:]
    times = pd.Series(route_dict[route]['time']).diff()[1:]-150
    times = (times-distances)
    times = [int(t) for t in times]
    #distances = np.array(distances).cumsum()
    times = np.array(times)

    y_ticks = [f'{name[0]} to {name[1]} - {datetime.timedelta(seconds=float(one_time))}' for one_time,name in zip(times,zip(route_dict[route]['current_names'], route_dict[route]['next_names']))]
    pyplot.yticks(ticks=times, labels=y_ticks)
    pyplot.plot(times, marker='o')
    #pyplot.plot(distances, marker='o')
    pyplot.show()